In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/finalised/csv/train_v2.csv')
X_test = pd.read_csv('/content/drive/MyDrive/fyp/fyp2/finalised/csv/test_v2.csv')

In [ ]:
X_train

,CamId,img1,img2,temp1,temp2,temp3,weather1,weather2,weather3
0,623,20130818_201151.jpg,20130818_211149.jpg,30.0,28.9,30.0,sunny,cloudy,sunny
1,8733,20131020_110640.jpg,20131020_160646.jpg,11.0,16.0,15.0,cloudy,cloudy,rainy
2,75,20130426_191257.jpg,20130426_221256.jpg,21.7,28.9,30.0,sunny,sunny,sunny
3,204,20130823_154951.jpg,20130823_181948.jpg,21.7,21.7,27.2,sunny,sunny,sunny
4,10917,20120121_085406.jpg,20120121_095406.jpg,1.0,2.0,2.0,cloudy,rainy,rainy
...,...,...,...,...,...,...,...,...,...
2203,260,20140803_184456.jpg,20140803_191501.jpg,30.0,30.0,30.0,cloudy,cloudy,sunny
2204,3297,20130107_133114.jpg,20130107_170123.jpg,5.0,5.0,5.0,cloudy,cloudy,rainy
2205,260,20130906_141501.jpg,20130906_201451.jpg,13.0,22.0,23.0,sunny,sunny,sunny
2206,7211,20120828_161005.jpg,20120828_164004.jpg,27.0,27.0,26.0,sunny,sunny,sunny


In [ ]:
X_test

,CamId,img1,img2,temp1,temp2,temp3,weather1,weather2,weather3
0,7211,20120526_064041.jpg,20120526_091002.jpg,16.0,16.0,18.0,sunny,sunny,sunny
1,4801,20110327_084003.jpg,20110327_094004.jpg,7.0,8.0,9.0,cloudy,rainy,rainy
2,4801,20111213_111002.jpg,20111213_104048.jpg,1.0,2.0,2.0,cloudy,rainy,rainy
3,75,20141010_174256.jpg,20141010_191301.jpg,25.0,25.0,32.2,sunny,sunny,sunny
4,684,20121105_174154.jpg,20121105_221148.jpg,10.0,12.8,11.7,sunny,sunny,sunny
...,...,...,...,...,...,...,...,...,...
923,5021,20130818_094627.jpg,20130818_124628.jpg,17.0,21.0,20.0,sunny,sunny,sunny
924,11160,20130822_102323.jpg,20130822_132312.jpg,13.0,20.0,23.0,sunny,sunny,sunny
925,4679,20140831_104013.jpg,20140831_101021.jpg,13.0,13.0,14.0,rainy,rainy,rainy
926,623,20120919_221154.jpg,20120919_021147.jpg,26.7,27.2,27.2,sunny,sunny,sunny


In [ ]:
test_df = pd.DataFrame(columns = ['img', 'next_temp'])
train_df = pd.DataFrame(columns = ['img', 'next_temp'])
path = '/content/drive/MyDrive/fyp/fyp2/image/'

for i in range(len(X_test)):
  test_df.loc[len(test_df)] = [str(path) + str(X_test['img1'].iloc[i]), X_test['temp2'].iloc[i]]
  test_df.loc[len(test_df)] = [str(path) + str(X_test['img2'].iloc[i]), X_test['temp3'].iloc[i]]

for i in range(len(X_train)):
  train_df.loc[len(train_df)] = [str(path) + str(X_train['img1'].iloc[i]), X_train['temp2'].iloc[i]]
  train_df.loc[len(train_df)] = [str(path) + str(X_train['img2'].iloc[i]), X_train['temp3'].iloc[i]]


In [ ]:
train_df

,img,next_temp
0,/content/drive/MyDrive/fyp/fyp2/image/20130818...,28.9
1,/content/drive/MyDrive/fyp/fyp2/image/20130818...,30.0
2,/content/drive/MyDrive/fyp/fyp2/image/20131020...,16.0
3,/content/drive/MyDrive/fyp/fyp2/image/20131020...,15.0
4,/content/drive/MyDrive/fyp/fyp2/image/20130426...,28.9
...,...,...
4411,/content/drive/MyDrive/fyp/fyp2/image/20130906...,23.0
4412,/content/drive/MyDrive/fyp/fyp2/image/20120828...,27.0
4413,/content/drive/MyDrive/fyp/fyp2/image/20120828...,26.0
4414,/content/drive/MyDrive/fyp/fyp2/image/20121029...,4.0


In [ ]:
from collections import deque 
import copy 

def filegenerator(df, temporal_length, temporal_stride):
  labels = list(df.next_temp)
  img_list = list(df.img)
  samples = deque()
  sample_count = 0

  for img in img_list:
    samples.append(img)
    if len(samples)== temporal_length: 
      samples_c = copy.deepcopy(samples)
      sample_count += 1

      temp_arr = [[np.float32(labels[0])], [np.float32(labels[1])]]
      yield samples_c, temp_arr
      labels = labels[2:]
      samples.clear()

In [ ]:
def seq_of_frames(df,length,stride):
  file_gen = filegenerator(df,length,stride)
  iterator = True
  data_list = []
  while iterator:
    try:
      X,y = next(file_gen)
      X = list(X) 
      data_list.append([X,y])
    except Exception as e:
      print("An exception has occured:",e)
      iterator = False
  return data_list

In [ ]:
training_data = seq_of_frames(train_df,2,2)
validation_data = seq_of_frames(test_df,2,2)

An exception has occured: 
An exception has occured: 


In [ ]:
from keras.utils import data_utils
import keras
import cv2

class DataGenerator(data_utils.Sequence):
  def __init__(self,data,batch_size,dim,is_autoencoder,shuffle):
    #Initializing the values
    self.dim = dim
    self.data  = data
    self.batch_size = batch_size
    self.list_IDs = np.arange(len(data))
    self.is_autoencoder = is_autoencoder
    self.shuffle = shuffle
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indexes = self.list_IDs
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __len__(self):
    return int(np.floor(len(self.data)/self.batch_size))

  def __getitem__(self, index):
    index = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    list_IDs_temp = [self.list_IDs[k] for k in index]
    X,y = self.__data_generation(list_IDs_temp)
    return X,y

  def __data_generation(self,list_IDs_temp):
    X_data = []
    y_data = []
    for i in list_IDs_temp:
      batch_samples = self.data[i][0]
      y = self.data[i][1]
      temp_data_list = []
      for img in batch_samples:
        try:
          image = cv2.imread(img)
          ext_img = cv2.resize(image,self.dim)
          ext_img = cv2.normalize(ext_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
          temp_data_list.append(ext_img)
        except Exception as e:
          print('image: ', img, 'Value error ',e)
          
      if len(temp_data_list) == 2:
        X_data.append(temp_data_list)
        y_data.append(y)

    X = np.array(X_data)
    y = np.array(y_data)
    if self.is_autoencoder == True:
      return X, X
    else:
      return X, y

In [ ]:
params = {
'batch_size':32,
'dim':(320, 180),
'is_autoencoder':False,
'shuffle':False }
train_gen = DataGenerator(training_data,**params)
valid_gen = DataGenerator(validation_data,**params)

In [ ]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed 
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import SGD

In [ ]:
def model():
  num_img = 2
  width = 320
  height = 180

  #cnn
  model = Sequential()
  model.add(TimeDistributed(VGG16(include_top=False),
                            input_shape=(num_img, height, width, 3)))

  model.add(TimeDistributed(Flatten()))

  #lstm
  model.add(LSTM(64, return_sequences=True, activation='sigmoid'))

  # dense layer
  model.add(Dense(64, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1))

  opt = SGD(lr=0.001, decay=0.025)
  model.compile(loss = 'mse', optimizer= opt, metrics=[metrics.RootMeanSquaredError()])


  return model

In [ ]:
import keras.backend as K

def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))  

In [ ]:
model = model()
filepath = "/content/drive/MyDrive/fyp/fyp2/finalised/code/conceptual checkpoint/version3/ep_{epoch:02d}-val_root_mean_squared_error_{val_root_mean_squared_error:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# no.of img, size,size, channel
model.build((2,180,320,3))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 2, 5, 10, 512)     14714688  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 25600)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 2, 64)             6570240   
_________________________________________________________________
dense (Dense)                (None, 2, 64)             4160      
_________________________________________________________________
dense_1 (Dense)              (None, 2, 16)             1040      
_________________________________________________________________
dense_2 (Dense)              (None, 2, 1)              17        
Total params: 21,290,145
Trainable params: 21,290,145
Non-trainable params: 0
____________________________________________

In [ ]:
history = model.fit(train_gen,epochs = 50,validation_data=valid_gen,use_multiprocessing=True,workers=4, callbacks=callbacks_list)

image:  /content/drive/MyDrive/fyp/fyp2/image/20130818_201151.jpg Value error  OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Epoch 1/50
69/69 [==============================] - 594s 8s/step - loss: 207.9413 - root_mean_squared_error: 14.2185 - val_loss: 117.6464 - val_root_mean_squared_error: 10.8465

Epoch 00001: saving model to /content/drive/MyDrive/Issac/fyp2/finalised/code/conceptual checkpoint/version3/ep_01-val_root_mean_squared_error_10.8465.hdf5
Epoch 2/50
69/69 [==============================] - 88s 1s/step - loss: 115.3630 - root_mean_squared_error: 10.7322 - val_loss: 90.8114 - val_root_mean_squared_error: 9.5295

Epoch 00002: saving model to /content/drive/MyDrive/Issac/fyp2/finalised/code/conceptual checkpoint/version3/ep_02-val_root_mean_squared_error_9.5295.hdf5
Epoch 3/50
69/69 [==============================] - 93s 1s/step - loss: 86.0530 - root_mean_squared_error: 9.2736 - val_loss: 6

In [ ]:
# save history
import pickle
with open('/content/drive/MyDrive/fyp/fyp2/finalised/code/conceptual checkpoint/version3/HistoryDict', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)


In [ ]:
min(history.history['val_root_mean_squared_error'])

4.650700569152832

In [ ]:
accuracy = pd.DataFrame({'train_loss': history.history['loss'], 
                         'train_rmse': history.history['val_root_mean_squared_error'],
                         'validation_loss': history.history['val_loss'],
                         'validation_rmse': history.history['val_root_mean_squared_error']}, 
                        columns = ['train_loss', 'train_rmse', 'validation_loss', 'validation_rmse'])
accuracy 

,train_loss,train_rmse,validation_loss,validation_rmse
0,152.679611,10.846494,117.646423,10.846494
1,111.112282,9.529503,90.811424,9.529503
2,80.641808,8.214019,67.470100,8.214019
3,59.731331,7.189651,51.691074,7.189651
4,48.918648,6.822597,46.547821,6.822597
5,40.976635,6.271281,39.328968,6.271281
6,34.854858,6.419770,41.213444,6.419770
7,30.083794,5.856873,34.302963,5.856873
8,26.052067,5.825997,33.942238,5.825997
9,22.901892,5.571833,31.045324,5.571833


In [ ]:
accuracy.to_csv('/content/drive/MyDrive/fyp/fyp2/finalised/code/conceptual checkpoint/version3/accuracy.csv', encoding='utf-8', index=False)
